In [2]:
!pip install simpy

  Obtaining dependency information for simpy from https://files.pythonhosted.org/packages/48/72/920ed1224c94a8a5a69e6c1275ac7fe4eb911ba8feffddf469f1629d47f3/simpy-4.1.1-py3-none-any.whl.metadata


In [3]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

class Library:
    def __init__(self, env, num_books, book_distribution, mean_arrival_time_users, scaling_factor):
        self.env = env
        self.book_distribution = book_distribution
        self.mean_arrival_time_users = mean_arrival_time_users
        self.books = {i: {'count': max(1, int(book_distribution[i] * scaling_factor/self.mean_arrival_time_users)), 'available': max(1, int(book_distribution[i] * scaling_factor/self.mean_arrival_time_users))} for i in range(num_books)}
        self.unavailability_histogram = {i: 0 for i in range(num_books)}
        self.waiting_queue = []
        self.waiting_history = []  # Verfolgt die Anzahl wartender Nutzer über die Zeit
        self.unavailable_count = 0
        self.user_arrivals = env.process(self.user_arrival())

    def use_book_on_site(self, book_id):
        if self.books[book_id]['available'] > 0:
            self.books[book_id]['available'] -= 1
            yield self.env.timeout(np.random.exponential(1))  # Nutzungsdauer
            self.books[book_id]['available'] += 1
        else:
            self.unavailable_count += 1
            self.unavailability_histogram[book_id] += 1  # Increment unavailability count
        self.check_waiting_queue()

    def borrow_book(self, book_id):
        if self.books[book_id]['available'] > 0:
            self.books[book_id]['available'] -= 1
            duration = np.random.randint(1, 10)  # Ausleihe für 1 bis 9 Tage
            yield self.env.timeout(duration)
            self.books[book_id]['available'] += 1
        #else: Do not count to avoid double-counting
        #    self.unavailable_count += 1
        #    self.unavailability_histogram[book_id] += 1  # Increment unavailability count
        self.check_waiting_queue()

    def request_book(self, book_id):
        if self.books[book_id]['available'] == 0:
            self.waiting_queue.append((self.env.now, book_id))
            self.unavailability_histogram[book_id] += 1  # Increment unavailability count
        else:
            self.env.process(self.borrow_book(book_id))
        self.waiting_history.append((self.env.now, len(self.waiting_queue)))  # Aufzeichnung der Warteschlangenlänge

    def check_waiting_queue(self):
        for request in sorted(self.waiting_queue, key=lambda x: x[0]):
            time_requested, book_id = request
            if self.books[book_id]['available'] > 0:
                self.env.process(self.borrow_book(book_id))
                self.waiting_queue.remove(request)

    def user_arrival(self):
        while True:
            book_id = np.random.choice(len(self.books), p=self.book_distribution)
            user_choice = np.random.choice(['on_site', 'borrow'], p=[0.5, 0.5])
            if user_choice == 'on_site':
                if self.books[book_id]['available'] > 0:
                    self.env.process(self.use_book_on_site(book_id))
                else:
                    self.unavailability_histogram[book_id] += 1  # Increment unavailability count
                    self.unavailable_count += 1
            elif user_choice == 'borrow':
                self.request_book(book_id)
            yield self.env.timeout(np.random.exponential(self.mean_arrival_time_users))  # Zeit zwischen Benutzerankünften
            self.waiting_history.append((self.env.now, len(self.waiting_queue)))  # Aktualisierung der Warteschlangenlänge

def simulate_library(num_books, mean_arrival_time_users, scaling_factor):
    env = simpy.Environment()
    num_days = 100
    book_distribution = np.random.pareto(a=2, size=num_books)
    book_distribution /= book_distribution.sum()    
    library = Library(env, num_books, book_distribution, mean_arrival_time_users, scaling_factor)
    env.run(until=num_days)

    # Visualisierung der wartenden Nutzer über die Zeit
    plt.figure(figsize=(10, 5))
    plt.plot([t[0] for t in library.waiting_history], [t[1] for t in library.waiting_history], label='Wartende Nutzer', color='red')
    plt.xlabel('Zeitschritte')
    plt.ylabel('Anzahl wartender Nutzer')
    plt.title('Anzahl wartender Nutzer über die Zeit')
    plt.legend()
    plt.show()

    # Visualisierung wie oft ein bestimmtes Buch nicht verfügbar war
    plt.figure(figsize=(10, 5))    
    plt.bar(range(num_books), [library.unavailability_histogram[i] for i in range(num_books)], color='blue')
    plt.plot(range(num_books), 100*book_distribution, color = 'red')
    plt.xlabel('Book ID')
    plt.ylabel('Anzahl wie oft nicht verfügbar')
    plt.title('Histogramm der Buchverfügbarkeit')
    plt.show()

    multi_run = []
    for k in range(10):
        env = simpy.Environment()
        library = Library(env, num_books, book_distribution, mean_arrival_time_users, scaling_factor)
        env.run(until=num_days)
        multi_run.append(np.max([library.unavailability_histogram[i] for i in range(num_books)]))
        
    # Visualisierung wie oft ein bestimmtes Buch nicht verfügbar war
    plt.figure(figsize=(10, 5))    
    plt.plot(multi_run, color = 'red')    
    plt.xlabel('Run ID')
    plt.ylabel('Max(Nicht verfügbare Bücher)')    
    plt.show()

# Parametereinstellungen
NUM_BOOKS = 100

#simulate_library(NUM_BOOKS, 0.1, 1.0)

from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets
# Interactive widgets to adjust parameters and rerun simulation
interact(simulate_library,         
         num_books=IntSlider(value=10, min=10, max=200, step=10, description='Number of books'),
         mean_arrival_time_users=FloatSlider(value=0.1, min=0.005, max=2.0, step=0.005, description='Mean arrival time'),
         scaling_factor=FloatSlider(value=1.0, min=1.0, max=100.0, step=0.1, description='Scaling Factor'))        

interactive(children=(IntSlider(value=10, description='Number of books', max=200, min=10, step=10), FloatSlide…

<function __main__.simulate_library(num_books, mean_arrival_time_users, scaling_factor)>